In [0]:
from pyspark.sql.functions import *
from configs.job_config import config

In [0]:
%sql
OPTIMIZE olist_db.silver.customers
ZORDER BY (customer_id); -- Joins with silver.orders and filter by customer; occasional state analysis; high-cardinal column

OPTIMIZE olist_db.silver.sellers
ZORDER BY (seller_id); -- Joins with the order_items; Seller performance analytics

OPTIMIZE olist_db.silver.products
ZORDER BY (product_id, product_category_name); -- Joins with the order_items; Category analytics

OPTIMIZE olist_db.silver.orders
ZORDER BY (order_purchase_timestamp, customer_id);

OPTIMIZE olist_db.silver.order_items
ZORDER BY (order_id, product_id);

OPTIMIZE olist_db.silver.order_payments
ZORDER BY (order_id);

OPTIMIZE olist_db.silver.geolocation
ZORDER BY (geolocation_zip_code_prefix);

OPTIMIZE olist_db.silver.order_reviews
ZORDER BY (order_id);

-- No optimize required for the product_category_translation table as this is a small lookup table

In [0]:
catalog_name = 'olist_db'
schema_name = 'silver'

for name in config['table_names']:
    table_name = catalog_name + "." + schema_name + "." + name
    spark.sql(f"""
                VACUUM {table_name} RETAIN 720 HOURS
              """)